## 本文主旨
- 这篇paper主要通过分组的方法提供序列的一种概率模型。In this paper, we present a probabilistic model for sequences via their segmentations.<br>
- 分组后的序列的概率由各个分组的概率相乘而得.The probability of a segmented sequence is calculated as the product of the probabilities of all its segments<br>
- 而各个分组的概率可以通过现成的工具，像RNN啊什么的，建模而成. seach segment is modeled using existing tools such as RNN<br>
- 由于同一个序列有多种不一样的分组方法，我们通过求和这些，通过不同的分组方法得到的分组序列的概率，来计算一个序列的概率。Since the segmentation of a sequence is usually unknown in advance, we sum over all valid segmentations to obtains the final probability for the sequence.

## Sequence modeling via segmentations
### Case 1: Mapping from non-sequence to sequence
- $S_y:$分类模式空间，含有所有可能的分组方法
- $a_{1:T_a} \in S_y:$表示一种分组方法
- $a_1,...,a_{T_a}：$表示各个分组，每个分组都以\$结尾。不允许出现仅含有\$的分组
- $\pi():$表示链接函数$\pi(a_{1:T_a})=y_{1:T}$
- $p(y_{1:T}|x) = \sum_{a_{1:T_a}\in S_y}p(a_{1:T_a}|x)=\sum_{a_{1:T_a}\in S_y}\prod^{T_a}_{t=1}p(a_t|x,\pi(a_{1:t-1}))$
- 我们使用LSTM或GRU加softmax来给$p(a_t|x,\pi(a_{1:t-1}))$（分组$a_t$）建模。使用输入x和$\pi(a_{1:t-1}$决定RNN的初始状态（？？？）
- 但是，$|S_y|$的大小是指数级的，我们没有办法直接求出上面提到的概率，这个问题，我们留待后面解决。
![non sequence to sequence](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/non_sequence_to_sequence.png)

### Case 2: Mapping from sequence to sequence：Sleep-WAke Network(SWAM)
- 输入为序列$x_{1:T'}$，输出和case 1的一样。
- 允许出现仅含有\$的分组，使得每个分组后的序列可以含有固定的长度T'
- $p(y_{1:T}|x) = \sum_{a_{1:T_a}\in S_y}p(a_{1:T_a}|x)=\sum_{a_{1:T_a}\in S_y}\prod^{T_a}_{t=1}p(a_t|x_t,\pi(a_{1:t-1}))$
- 同样的我们可以使用LSTM或GRU加softmax来给$p(a_t|x_t,\pi(a_{1:t-1}))$。使用输入$x_t$和$\pi(a_{1:t-1})$决定RNN的初始状态
- 同样的存在和case 1相同的问题
 ![seq to seq](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/seq_to_seq.png)

### Carrying over information across segments
- 受sequence transducer(2012 Graves)的启发，我们使用分离的（额外的）RNN来为$\pi(a_{1:t-1}）$建模。使用这个RNN的中间层的输出和输入$x_t$决定分组$a_t$的RNN模型的初值状态
![carry infos](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/carry_info.png)

### related work
好多啊

## Forward, backward and decoding
### Forward and backward propagations
- 定义forward backward概率：<br>
$\alpha_t(j)=p(y_{1:j}|x_{1:t})$<br>
$\beta_t(j)=p(y_{j+1,T}|x_{t+1:T'},y_{1:j})$<br>
t,j分别代表原序列和目标序列的分割点。
- Using $\alpha\beta$， we can verify the following, for any t=0, ..., T',<br>
$p(y_{1:T}|x_{1:T'})=\sum_{j=0}^T\alpha_t(j)\beta_t(j)$<br>
表示当t固定的时候，目标序列的所有可能的二类分组的概率之和
- 存在特殊情况：$p(y_{1:T}|x_{1:T'})= \alpha_{T'}(T)=\beta_0(0)$
- 更进一步，$\alpha\beta$具有以下递归特性：<br>
$\alpha_t(j)=\sum_{j'=0}^j\alpha_{t-1}(j')p(y_{j'+1:j}|x_t)$<br>
$\beta_t(j)=\sum_{j'=j}^T\beta_{t+1}(j')p(y_{j+1:j'}|x_{t+1})$
- as we discussed before, $p(y_{j'+1:j}|x_t)$ is modeled using an RNN with a softmax probability function. Given initial conditions $\alpha_0(0)=1$ and $\beta_{T'}(T) = 1$, we can efficiently compute the probability of the entire output $p(y_{1:T}|x_{1:T'})$

T'是输入序列的长度，T是输出的。

## 补充
### 动态编程
动态编程是在序列分析中经常使用的一种算法技术。在可以使用递归，但因为递归重复解决相同的子问题造成效率低下的时候，则可以采用动态编程。例如，请看斐波纳契（Fibonacci）序列：0, 1, 1, 2, 3, 5, 8, 13, ... 第一个和第二个斐波纳契数字分别定义为0和1。第 n个斐波纳契数字是前两个斐波纳契数字的和。<br>
动态编程是一种在数学和电脑科学中使用的，用于求解包含重叠子问题的最优化问题的方法。其基本思想是，将原问题分解为相似的子问题，在求解的过程中通过子问题的解求出原问题的解。动态编程的思想是多种演算法的基础，被广泛应用于电脑科学和工程领域。比较著名的应用实例有：求解最短路径问题，背包问题，专案管理，网路流优化等。<br>